## Import our modules

In [1]:
import pandas as pd 
import numpy as np
import warnings
# current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
import gc
import datetime

##  Loading our data

In [2]:
data = pd.read_excel("BankNiftyFutures_Data5Min.xlsx", encoding = 'ISO-8859-1')

## Data Preprocessing

In [3]:
data.loc[:,'Time1'] = pd.to_datetime(data.Time.astype(str))
data['Time15'] = data.Time1.dt.round('15T').dt.time.map(lambda s: str(s))
data.loc[:,'Date'] = pd.to_datetime(data.Date.astype(str)+' '+data.Time15.astype(str))
data1 = data[['Date','Open 5', 'High 5','Low 5', 'Close 5', 'Volume 5']]

In [6]:
def calculatedata(data):
    data['High'] = data['High 5'].max()
    data['Low'] = data['Low 5'].min()
    data['Volume'] = data['Volume 5'].sum()
    
    data['Open'] = data['Open 5'].iloc[0]
    data['Close'] = data['Close 5'].iloc[-1]
    
    return data

data2 = data1.groupby('Date').apply(calculatedata)
data3 = data2[['Date','Open','High','Low','Close','Volume']].drop_duplicates().reset_index()

# Calculation of VWAP

In [11]:
def calculateVwap(data):
    data['TP'] = (data['High']+data['Low']+data['Close'])/3.0
    data['TradedValue']  = data['TP']*data['Volume']
    data['CumVolume'] = data['Volume'].cumsum()
    data['CumTradedValue'] = data['TradedValue'].cumsum()
    data['VWAP'] = data['CumTradedValue'] /data['CumVolume']
    return data

data4 = data3.groupby('Date').apply(calculateVwap)

In [16]:
data4.head()

,Date,Open,High,Low,Close,Volume,TP,TradedValue,CumVolume,CumTradedValue,VWAP
0,2015-01-01 09:15:00,18861.3496,18867.9492,18793.0000,18808.3496,84825,18823.099600,1.596669e+09,84825,1.596669e+09,18823.099600
1,2015-01-01 09:30:00,18808.0000,18835.0000,18802.1992,18810.0000,85950,18815.733067,1.617212e+09,85950,1.617212e+09,18815.733067
2,2015-01-01 09:45:00,18805.0000,18839.0508,18775.0000,18837.0000,108925,18817.016933,2.049644e+09,108925,2.049644e+09,18817.016933
3,2015-01-01 10:00:00,18837.0000,18858.5996,18830.0000,18853.8008,64250,18847.466800,1.210950e+09,64250,1.210950e+09,18847.466800
4,2015-01-01 10:15:00,18852.0000,18858.0000,18830.0000,18841.0000,37350,18843.000000,7.037860e+08,37350,7.037860e+08,18843.000000


In [15]:
data4.drop('index',axis = 1, inplace= True)
data4['Time'] = data4['Date'].dt.time
data4['Date'] = data4['Date'].dt.date

## Trade process

In [64]:
Trade1 = pd.DataFrame(columns=["Date","BuyAmnt","SellAmnt",'Profit/Loss'])

for i in range(0, len(data4)):
    if (data4.loc[i,"Time"]>=datetime.datetime.strptime('10:00:00', '%H:%M:%S').time()) & (data4.loc[i,"Time"]<datetime.datetime.strptime('15:00:00', '%H:%M:%S').time()):
        if (data4['Close'][i] >= data4['VWAP'][i]):
            BuyAmnt = data4.loc[i,"Close"] +0.05
            date = data4.loc[i,"Date"]
            continue
        elif (data4['Close'][i] <= data4['VWAP'][i]):
            SellAmnt = data4.loc[i,"Close"] - 0.05
            date = data4.loc[i,"Date"]
            
    #exit the trade:
    elif (data4.loc[i,"Time"]>=datetime.datetime.strptime('15:15:00', '%H:%M:%S').time()):
            SellAmnt = data4.loc[i,"Close"]
            date = data4.loc[i,"Date"]

    
            Trade1 = Trade1.append({"Date":data4.loc[i,"Date"],
                                    "BuyAmnt":BuyAmnt,
                                    "SellAmnt":SellAmnt,
                                    "Profit/Loss":(SellAmnt-BuyAmnt)},
                                    ignore_index=True)
            
            


In [76]:
Trade1.loc[:,'Date'] = pd.to_datetime(Trade1.Date.astype(str))
Trade1['year'] = Trade1['Date'].dt.year
Trade1['month']=Trade1['Date'].dt.month

## Calculating Profit/Loss for each year

In [74]:
Trade1.groupby(['year'])['Profit/Loss'].sum() 

year
2015   -16400.7528
2016    -9773.2604
2017    -5308.0716
2018   -11170.9292
2019   -19893.5992
Name: Profit/Loss, dtype: float64

## According to above analysis, 2017 was the year in which loss was minimum.

#### NOTE : There seems some issue while handling the stoploss. This can be rectified under your guidance.